# HappyRobot Load Management API Testing

This notebook provides examples of how to interact with the HappyRobot backend API for load management.

## Features:
- Create new loads
- Update existing loads
- Assign loads (set status to agreed)
- List all loads
- Get specific load details
- Delete loads

## Prerequisites:
- Backend running on port 8000
- Valid API key (if authentication is enabled)


In [24]:
# Import required libraries
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, Any, Optional
import uuid


## Configuration

Set your backend URL and API key here:


In [25]:
# Configuration
BASE_URL = "http://localhost:8000"  # Change this to your backend URL
API_KEY = "HapRob-OTVHhErcXLu2eKkUMP6lDtrd8UNi61KZo4FvGALqem0NoJO1uWlz7OywCN0BNoNaG2x5Y"  # Replace with your actual API key

# Headers for API requests
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "X-API-Key": API_KEY  # API key header
}

print(f"Backend URL: {BASE_URL}")
print(f"API Key: {API_KEY[:10]}..." if len(API_KEY) > 10 else f"API Key: {API_KEY}")


Backend URL: http://localhost:8000
API Key: HapRob-OTV...


## Helper Functions


In [26]:
def make_request(method: str, endpoint: str, data: Optional[Dict] = None) -> Dict[str, Any]:
    """
    Make an API request to the backend
    
    Args:
        method: HTTP method (GET, POST, PATCH, DELETE)
        endpoint: API endpoint (e.g., '/shipments')
        data: Request data for POST/PATCH requests
    
    Returns:
        Response data as dictionary
    """
    url = f"{BASE_URL}{endpoint}"
    
    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=headers)
        elif method.upper() == "POST":
            response = requests.post(url, headers=headers, json=data)
        elif method.upper() == "PATCH":
            response = requests.patch(url, headers=headers, json=data)
        elif method.upper() == "DELETE":
            response = requests.delete(url, headers=headers)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
        
        # Check if request was successful
        if response.status_code in [200, 201, 204]:
            if response.status_code == 204:  # No content
                return {"success": True, "message": "Operation completed successfully"}
            return {"success": True, "data": response.json()}
        else:
            return {
                "success": False, 
                "error": f"HTTP {response.status_code}",
                "message": response.text
            }
            
    except requests.exceptions.RequestException as e:
        return {"success": False, "error": "Request failed", "message": str(e)}

def health_check() -> bool:
    """Check if the backend is healthy"""
    result = make_request("GET", "/health")
    if result["success"]:
        print("✅ Backend is healthy")
        return True
    else:
        print(f"❌ Backend health check failed: {result.get('message', 'Unknown error')}")
        return False

def print_response(result: Dict[str, Any], title: str = "Response"):
    """Pretty print API response"""
    print(f"\n{'='*50}")
    print(f"{title}")
    print(f"{'='*50}")
    
    if result["success"]:
        print("✅ Success!")
        if "data" in result:
            print(json.dumps(result["data"], indent=2, default=str))
        elif "message" in result:
            print(result["message"])
    else:
        print("❌ Failed!")
        print(f"Error: {result.get('error', 'Unknown error')}")
        print(f"Message: {result.get('message', 'No message')}")
    print(f"{'='*50}\n")


## 1. Health Check

First, let's verify that the backend is running and accessible:


In [27]:
# Check backend health
health_check()


✅ Backend is healthy


True

## 2. Create a New Load

Create a new load with all the required and optional fields:


In [43]:
# Create a new load
def create_load(load_data: Dict[str, Any]) -> Dict[str, Any]:
    """Create a new load"""
    return make_request("POST", "/shipments", load_data)

# Example load data
new_load = {
    "load_id": f"LD-NOTEBOOK-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    "origin": "Los Angeles, CA",
    "destination": "New York, NY",
    "pickup_datetime": (datetime.now() + timedelta(days=1)).isoformat() + "Z",
    "delivery_datetime": (datetime.now() + timedelta(days=3)).isoformat() + "Z",
    "equipment_type": "Dry Van",
    "loadboard_rate": 2500.50,
    "weight": 1500.0,
    "commodity_type": "Electronics",
    "num_of_pieces": 100,
    "miles": 2800.0,
    "dimensions": "48x40x60 in",
    "notes": "Created from Jupyter notebook - handle with care"
}

print("Creating new load...")
print(f"Load ID: {new_load['load_id']}")
print(f"Route: {new_load['origin']} → {new_load['destination']}")

result = create_load(new_load)
print_response(result, "Create Load")


Creating new load...
Load ID: LD-NOTEBOOK-20250911-051318
Route: Los Angeles, CA → New York, NY

Create Load
✅ Success!
{
  "load_id": "LD-NOTEBOOK-20250911-051318",
  "origin": "Los Angeles, CA",
  "destination": "New York, NY",
  "pickup_datetime": "2025-09-12T05:13:18.234613Z",
  "delivery_datetime": "2025-09-14T05:13:18.234613Z",
  "equipment_type": "Dry Van",
  "loadboard_rate": 2500.5,
  "notes": "Created from Jupyter notebook - handle with care",
  "weight": 1500.0,
  "commodity_type": "Electronics",
  "num_of_pieces": 100,
  "miles": 2800.0,
  "dimensions": "48x40x60 in",
  "agreed_price": null,
  "carrier_description": null,
  "id": "75d34ccf-9a76-4084-a773-58f055ae4435",
  "status": "pending",
  "created_at": "2025-09-11T03:13:18.237651",
  "updated_at": "2025-09-11T03:13:18.237653"
}



## 3. Assign a Load (Set Status to Agreed)

This is the specific operation you requested - changing a load status to "assigned" (agreed) with the required agreed_price and carrier_description:


In [29]:
# First, let's get all loads to see what we have
def get_all_loads() -> Dict[str, Any]:
    """Get all loads"""
    return make_request("GET", "/shipments")

# Get all loads
all_loads_result = get_all_loads()
print_response(all_loads_result, "All Loads")

# Find a load to assign (preferably one that's not already assigned)
if all_loads_result["success"] and all_loads_result["data"]:
    loads = all_loads_result["data"]
    
    # Find a pending load to assign
    pending_loads = [load for load in loads if load["status"] == "pending"]
    
    if pending_loads:
        load_to_assign = pending_loads[0]
        print(f"\n🎯 Found load to assign: {load_to_assign['load_id']}")
        print(f"   Route: {load_to_assign['origin']} → {load_to_assign['destination']}")
        print(f"   Current status: {load_to_assign['status']}")
        
        # Assign the load with required fields
        assign_data = {
            "status": "agreed",
            "agreed_price": 2800.00,  # Required when status is agreed
            "carrier_description": "ABC Transport Co."  # Required when status is agreed
        }
        
        print(f"\n📝 Assigning load with:")
        print(f"   Agreed Price: ${assign_data['agreed_price']}")
        print(f"   Carrier: {assign_data['carrier_description']}")
        
        # Make the PATCH request to assign the load
        assign_result = make_request("PATCH", f"/shipments/{load_to_assign['id']}", assign_data)
        print_response(assign_result, "Assign Load")
        
    else:
        print("\n⚠️ No pending loads found to assign")
        print("All loads are already assigned or no loads exist")
        
        # Show current loads status
        if loads:
            print("\n📊 Current loads status:")
            for load in loads:
                print(f"   {load['load_id']}: {load['status']}")
else:
    print("❌ Could not retrieve loads")



All Loads
✅ Success!
[
  {
    "load_id": "LD-NOTEBOOK-20250911-050709",
    "origin": "Los Angeles, CA",
    "destination": "New York, NY",
    "pickup_datetime": "2025-09-12T05:07:09.730132Z",
    "delivery_datetime": "2025-09-14T05:07:09.730132Z",
    "equipment_type": "Dry Van",
    "loadboard_rate": 2500.5,
    "notes": "Created from Jupyter notebook - handle with care",
    "weight": 1500.0,
    "commodity_type": "Electronics",
    "num_of_pieces": 100,
    "miles": 2800.0,
    "dimensions": "48x40x60 in",
    "agreed_price": null,
    "carrier_description": null,
    "id": "0da37dd3-bfb9-41f4-8780-d9651ff9e83a",
    "status": "pending",
    "created_at": "2025-09-11T03:07:09.736204",
    "updated_at": "2025-09-11T03:07:09.736205"
  },
  {
    "load_id": "LD-NOTEBOOK-20250911-050142",
    "origin": "Los Angeles, CA",
    "destination": "New York, NY",
    "pickup_datetime": "2025-09-12T05:01:42.335220Z",
    "delivery_datetime": "2025-09-14T05:01:42.335220Z",
    "equipment_type

In [ ]:
# First, let's verify the load exists and check its current status
load_id = "LD-NOTEBOOK-20250911-051318"

print(f"🔍 Checking if load {load_id} exists...")

# Get all loads to find the one we want
all_loads_result = get_all_loads()

if all_loads_result["success"] and all_loads_result["data"]:
    loads = all_loads_result["data"]
    
    # Find the specific load
    target_load = None
    for load in loads:
        if load["load_id"] == load_id:
            target_load = load
            break
    
    if target_load:
        print(f"✅ Found load: {target_load['load_id']}")
        print(f"   Current status: {target_load['status']}")
        print(f"   Route: {target_load['origin']} → {target_load['destination']}")
        print(f"   Loadboard rate: ${target_load.get('loadboard_rate', 'N/A')}")
        
        if target_load['status'] == 'pending':
            print(f"\n🎯 Load is pending - ready to assign!")
            
            # Now assign the load
            print(f"\n🚀 Assigning load {load_id}...")
            
            url = f"http://localhost:8000/shipments/{target_load['id']}"
            headers = {
                "Content-Type": "application/json",
                "X-API-Key": "HapRob-OTVHhErcXLu2eKkUMP6lDtrd8UNi61KZo4FvGALqem0NoJO1uWlz7OywCN0BNoNaG2x5Y"
            }
            data = {
                "status": "agreed",
                "agreed_price": 2800.0,
                "carrier_description": "ABC Transport Co."
            }
            
            try:
                response = requests.patch(url, headers=headers, json=data)
                
                print(f"📊 Response Status Code: {response.status_code}")
                
                if response.status_code == 200:
                    print("\n✅ Success! Load assigned successfully:")
                    updated_load = response.json()
                    print(f"   New status: {updated_load['status']}")
                    print(f"   Agreed price: ${updated_load.get('agreed_price', 'N/A')}")
                    print(f"   Carrier: {updated_load.get('carrier_description', 'N/A')}")
                else:
                    print(f"\n❌ Request failed with status code: {response.status_code}")
                    print(f"Response: {response.text}")
                    
            except requests.exceptions.RequestException as e:
                print(f"\n💥 Request failed: {str(e)}")
                
        elif target_load['status'] == 'agreed':
            print(f"\n⚠️ Load is already assigned (agreed)")
            print(f"   Agreed price: ${target_load.get('agreed_price', 'N/A')}")
            print(f"   Carrier: {target_load.get('carrier_description', 'N/A')}")
        else:
            print(f"\n❓ Load has unexpected status: {target_load['status']}")
    else:
        print(f"❌ Load {load_id} not found!")
        print("\n📋 Available loads:")
        for load in loads:
            print(f"   {load['load_id']}: {load['status']}")
else:
    print("❌ Could not retrieve loads from the API")


🔍 Checking if load LD-2025-0008 exists...
✅ Found load: LD-2025-0008
   Current status: agreed
   Route: Hamburg → Stockholm
   Loadboard rate: $2800.0

⚠️ Load is already assigned (agreed)
   Agreed price: $2800.0
   Carrier: ABC Transport Co.
